In [36]:
import nltk
import pickle
from nltk.stem.lancaster import LancasterStemmer
import numpy
import tflearn
import tensorflow as tf
import random
import json
import numpy as np

In [37]:
stemmer = LancasterStemmer()

In [38]:
# Load intents from JSON file with explicit encoding
with open('intents1.json', encoding='utf-8') as file:
    data = json.load(file)

In [39]:
words = []
labels = []
docs_x = []
docs_y = []

In [40]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [41]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [42]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

In [43]:
for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)



In [44]:
training = np.array(training)
output = np.array(output)
with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

tf.compat.v1.reset_default_graph()

In [45]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

In [52]:
model = tflearn.DNN(net)

#model.load("model.tflearn")

model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")


Training Step: 4999  | time: 0.061s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 0.0000 -- iter: 32/40
Training Step: 5000  | time: 0.064s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 0.0000 -- iter: 40/40
--
INFO:tensorflow:e:\AI-SIC\aaavi\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [53]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

In [55]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
Hello!
I cannot age.
An advance ai for your help.
Full form for my name is 'Artificial Assistant Available Virtually'
You can call me AAAVI.
You should take rest and watch something funny
To be honest i am student project ! but i think i am helpful
Yeah, I know that love is a beautiful emotion, but I suggest you just go and ask him/her out. 
Any time!
I am new born Bot!
Hi there, how can I help?
